# Using `Mini-instruct` on SageMaker through Model Packages

Developed by <a href="https://lighton.ai/"/>LightOn</a>, `mini-instruct` is a powerful, multilingual AI model with 40B parameters trained on high-quality data from a variety of sources.
It is designed to understand natural language and respond to instructions tailored to your needs. It works great in consumer products, such as chatbots, voice assistants, and smart appliances. It also has broad applications in the enterprise, such as natural language generation for automated customer service or agent assist for customer support.

If you want to know more about the best ways to prompt large language models, you can have a look at the <a href="https://lightonai.github.io/paradigm-docs/guides/prompt">documentation</a>. If you are familiar with the prompting literature, advanced techniques like <a href="https://arxiv.org/abs/2201.11903">Chain of Thought</a> also work with it.

Summarizing is as easy as adding *Summary :* after the relevant text snippet, simply change it to *Keywords :* to perform keywords extraction instead. The only limit is what you can express in text.

For example

>Extract the key words from the following article: Corium is a metallic and mineral magma consisting of the molten elements of a nuclear reactor core, and then the minerals it may absorb as it travels. The term "corium" is a neologism formed from core, followed by the suffix -ium, which is present in the names of many elements in the periodic table of elements: lithium, calcium, uranium, plutonium, helium, strontium, etc. Initially made up of the nuclear fuel (mainly enriched uranium oxide), the elements of the fuel assembly and the various pieces of core equipment (control rods, instrumentation) or the wall of the reactor vessel with which it comes into contact, it forms at very high temperature (around 3,000°C, the melting temperature of uranium oxide) when the core is no longer cooled, as during nuclear accidents such as those at Three Mile Island, Chernobyl or Fukushima.
>
>Keywords: corium, nuclear reactor, core meltdown

This sample notebook shows you how to deploy `mini-instruct` using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.

## Contents:
1. [Select model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Select to the model package
Confirm that you received this notebook from model catalog on SageMaker AWS Management Console.

In [1]:
# Mapping for Model Packages (initially only us-east-1 and eu-west-1 is supported)
model_package_map = {
    "us-east-1": "arn:aws:sagemaker:us-east-1:372939818206:model-package/mini-instruct",
    "eu-west-1": "arn:aws:sagemaker:eu-west-1:372939818206:model-package/mini-instruct"
}

In [4]:
import json
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import sagemaker as sage
import boto3

In [5]:
region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")

model_package_arn = model_package_map[region]

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()

runtime_sm_client = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [ ]:
model_name = "mini-instruct"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.p4d.24xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name, model_data_download_timeout=3600)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

For more details on the parameters of the endpoint Create, see the <a href="https://lightonai.github.io/paradigm-docs/api/endpoints/create">docs</a>.

In [ ]:
payload = {
    "data":
    {
        "text": "Generate a list of ideas for articles on watercolour.\n1. Watercolour in history.\n2.",
        "params": {
            "n_tokens": 43,
            "seed": 0
        }
    },
    "endpoint":"llm/create"
}

<Add code snippet that shows the payload contents>

### C. Perform real-time inference

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=model_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)

output = json.loads(response["Body"].read().decode("utf8"))

### D. Visualize output

In [ ]:
print(f"{payload['text']} 🤖 {output['outputs'][0]['completions'][0]['output_text']}")

You have seen above how to use JSON payloads to invoke an endpoint, now let's use the Python SDK. Using the Python SDK provides a better, more streamlined user experience, and it is recommeneded for experimentation.

The Python SDK takes care for you of formatting the input, calling the endpoint, and unpacking the output. There is one class per endpoint: `Create`, `Analyse`, `Select`, `Embed`, `Compare` and `Tokenize`.

In [ ]:
from paradigm_client.remote_model import RemoteModel
from paradigm_client.communicator import SagemakerCommunicator

comm = SagemakerCommunicator(model_name)
model = RemoteModel(model_name=model_name, comm=comm)

#### Using Create for sentiment analysis

You should describe the task you want to carry out in French, for example in the following: *Détermine si ces commentaires expriment des avis positifs, négatifs ou mitigés.*

In addition, providing examples improves the performance of the model.

In [ ]:
import re

few_shot_prompt = examples = f"""Determine whether these comments are positive, negative or neutral.
----------
Comment: \"Sorry but I have no compliments for this garage. I was on Saint-Etienne I was punctured with well on a nail in the tyre, the employee said to me that he could not help me, that it was necessary that I inflate my tyre to 2.4 I went down again to Roanne it was necessary that I stop halfway to inflate my tyre. Look for the error".
This comment expresses a negative opinion.
----------
Comment: \I had to stop midway to inflate my tyres, but I had to stop halfway to inflate my tyres. I had to stop halfway to inflate my tyres.
This comment expresses a positive opinion.
----------
Comment: \"It's OK but the employees are not necessarily all very well qualified.
This comment expresses a neutral opinion.
----------
Comment: \"Complicated and burdensome battery warranty, abusive marketing. If it fails, they change it for you, but they charge you for the replacement labour!
This comment expresses a negative opinion.
----------
Comment: \"Good welcome, appointment time respected and the price of the service much cheaper than in a car brand garage, next service or any other service I will do it in Norauto, very good garage."
This comment expresses a positive opinion.
----------
Comment: \"Excellent sign. The staff are competent and pleasant. I'm bookmarking this address."
This comment expresses an opinion"""

create_response = model.create(prompt=few_shot_prompt, stop_regex=re.compile(r"[.|\n]"))
print(f"{few_shot_prompt}🤖{create_response.completions[0].output_text}")

#### Using Select for review classification

In [ ]:
prompt = """Given a comment and the category to which it belongs, determine the most appropriate sub-category.

Comment: \"Came in to get my flat tire repaired, the advisor was quick to note that the tyre is not repairable. She offered me equivalent tyres and made an appointment for the afternoon, as on a Saturday morning, the centre was saturated. When I arrived at the appointment, I was quickly picked up and 30 minutes later, the vehicle was ready - I was warned by SMS. I was very pleased with the service and the quality of the service I received.
Category:"""

In [ ]:
select_response = model.select(reference=prompt, candidates=["accueil et conseils", "prise rendez-vous"])
print(f"{prompt}🤖{select_response.best}")

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()